In [1]:
from random import shuffle
import glob, h5py, pickle, cv2
import numpy as np
from IPython.display import clear_output
from pandas import read_csv
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from collections import Counter
width = 128
height = 128
channels = 3
trainpath = '/mnt/HDD04/Projection_data/OUTPUTS/Cascade_AWR2243/microDoppler/*.png'
sub = 'multi_target'
filename = 'datasets/'+sub+'.pkl'
print('File to be created: ' + filename)

File to be created: datasets/multi_target.pkl


In [2]:
# load single act images
train_files2 = sorted(glob.glob(trainpath))
print(len(train_files2))
train_files = []
fname_ls = []
multi_angs_ls = []
for i, file in enumerate(train_files2):
    
    fname = train_files2[i].split('/')[-1]
#     print(train_files2[i])
    angles = fname.split('_')[2][3:]
    plusses = angles.rfind('+')
    minuses = angles.rfind('-')

    if plusses == -1 and minuses == -1:
        continue
        all_ang_ids = int(angles)
        angs = all_ang_ids
    elif plusses == -1 and minuses != -1:
        all_ang_ids = sorted([minuses, len(angles)+1])
    elif plusses != -1 and minuses == -1:
        all_ang_ids = sorted([plusses, len(angles)+1])
    else:
        all_ang_ids = sorted([plusses, minuses, len(angles)+1])

    if isinstance(all_ang_ids, list):
        if all_ang_ids[0] == 0:
            angs = np.zeros((len(all_ang_ids) - 1,), dtype=int)
            for i in range(len(all_ang_ids) - 1):
                angs[i] = int(angles[all_ang_ids[i]:all_ang_ids[i+1]]) # .astype(int)
        else:
            angs = np.zeros((len(all_ang_ids),), dtype=int)
            angs[0] = int(angles[:all_ang_ids[0]])
            for i in range(len(all_ang_ids) - 1):
                angs[i+1] = int(angles[all_ang_ids[i]:all_ang_ids[i+1]]) # .astype(int)
              
    if len(angs) == 1:
        continue
    
    multi_angs_ls.append(angs) 
#     print(file)
    train_files.append(file)
print('Num. of files: '+str(len(train_files)))
for i in range(len(train_files)):    
    print(str(i+1)+'.'+train_files[i])

679
Num. of files: 21
1./mnt/HDD04/Projection_data/OUTPUTS/Cascade_AWR2243/microDoppler/pCemreRonyShobbo_class1-1-2_ang-45+45-0_iter1.png
2./mnt/HDD04/Projection_data/OUTPUTS/Cascade_AWR2243/microDoppler/pCemreRonyShobbo_class1-1-2_ang-45+45-0_iter2.png
3./mnt/HDD04/Projection_data/OUTPUTS/Cascade_AWR2243/microDoppler/pCemreRonyShobbo_class1-1-2_ang-45+45-0_iter3.png
4./mnt/HDD04/Projection_data/OUTPUTS/Cascade_AWR2243/microDoppler/pCemreRonyShobbo_class1-1-6_ang-45+45-0_iter1.png
5./mnt/HDD04/Projection_data/OUTPUTS/Cascade_AWR2243/microDoppler/pCemreRonyShobbo_class1-1-6_ang-45+45-0_iter2.png
6./mnt/HDD04/Projection_data/OUTPUTS/Cascade_AWR2243/microDoppler/pCemreRonyShobbo_class1-1-6_ang-45+45-0_iter3.png
7./mnt/HDD04/Projection_data/OUTPUTS/Cascade_AWR2243/microDoppler/pCemreRony_class1-1_ang+45-45_iter1.png
8./mnt/HDD04/Projection_data/OUTPUTS/Cascade_AWR2243/microDoppler/pCemreRony_class1-1_ang+45-45_iter2.png
9./mnt/HDD04/Projection_data/OUTPUTS/Cascade_AWR2243/microDoppler/pCem

In [3]:
labels = []
files = []
class_ls = []
cnt = 1
for i, file in enumerate(train_files):
    clear_output(wait=True)
    print('Loading training labels #'+str(cnt))
    fname = file.split('/')[-1]
    classes = [int(i)-1 if i != '6' else 2 for i in fname.split('_')[1][5:].split('-')]
    class_ls.append(classes)
    labels.append(classes)
    files.append(file)
    cnt += 1
    
labels = np.array(labels)
print(labels.shape)

Loading training labels #21
(21,)


In [4]:
for i in labels:
    print(i)

[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 2]
[0, 0, 2]
[0, 0, 2]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 1]
[0, 1]
[0, 1]
[0, 1]
[0, 1]
[0, 1]
[0, 1]
[2, 0]
[2, 0]
[2, 0]
[2, 0]


In [5]:
cnt = 1
images = []

for image in files:
    img = cv2.imread(image)
    img = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC)# resize to (128,128)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 load images as BGR, convert it to RGB
    clear_output(wait=True)
    print('Loading image '+str(cnt)+'/'+str(len(files))+ ' '+image)
    images.append(img)  
    cnt += 1
    
images = np.swapaxes(np.array(images), 1, 2).reshape(len(files), width, height, channels)/255.
images.shape

Loading image 21/21 /mnt/HDD04/Projection_data/OUTPUTS/Cascade_AWR2243/microDoppler/pCemreRony_class6-1_ang0+45_iter4.png


(21, 128, 128, 3)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=1)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(16, 128, 128, 3)
(16,)
(5, 128, 128, 3)
(5,)


In [7]:
data = [x_train, x_test, y_train, y_test]
with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(data, output, pickle.HIGHEST_PROTOCOL)
print(filename+' created.')

datasets/multi_target.pkl created.
